In [ ]:
# !pip install datasets
# !pip install evaluate
# !pip install rouge_score
!pip install transformers datasets evaluate rouge_score

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import numpy as np
import evaluate
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# **Load Dataset**

In [ ]:
ds_full = load_dataset('multi_news')
ds_train = ds_full['train']
ds_5000 = ds_train.shuffle(seed=42).select(range(5000))
dataset = ds_5000.train_test_split(test_size=0.2)

In [4]:
print(dataset.shape)

{'train': (4000, 2), 'test': (1000, 2)}


# **Preprocess Data**

In [ ]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Tokenize the labels
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Assuming 'dataset' is already loaded and defined
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [7]:
# checkpoint = "google-t5/t5-small" # We need an encoder-decoder model since we're going text-text
# tokenizer = AutoTokenizer.from_pretrained(checkpoint) # Use the right tokenizer
# prefix = "summarize: " # This is a multipurpose model - we need to attach a task to tell it what we want

In [8]:
# def preprocess_function(examples):
#     inputs = [prefix + doc for doc in examples["document"]] # add summarize to documents
#     model_inputs = tokenizer(inputs, max_length=1024, truncation=True) # tokenize inputs
#     labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True) # tokenize outputs

#     model_inputs["labels"] = labels["input_ids"] # match up the text and summary. Specific to this application
#     return model_inputs

In [9]:
# tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [10]:
# from transformers import DataCollatorForSeq2Seq
# # data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint) # code when using Pytorch
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

# **Define Pipeline Evaluation Metrics**

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# **Define Model**

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [15]:
# Original training args:

training_args = Seq2SeqTrainingArguments(
    output_dir="multi_news_train_4000t_t5-small_summary_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01, # regularization in optimizer
    save_total_limit=3, # maximum number of versions to have saved
    num_train_epochs=3,
    predict_with_generate= True
)

In [19]:
# # Attempted training args:
# # unable to run without crashing session

# training_args = Seq2SeqTrainingArguments(
#     output_dir="multi_news_train_4000arg_t5-small_summary_model",
#     save_strategy="epoch",  # Saves at the end of each epoch
#     save_total_limit=5,  # Increase if you have enough storage and want more checkpoints
#     evaluation_strategy="steps",  # Change from 'epoch' to evaluate more frequently
#     eval_steps=400,  # Number of steps to run evaluation
#     learning_rate=2e-5,
#     per_device_train_batch_size=8, # increase to provide more accurate estimate of gradient
#     per_device_eval_batch_size=8,
#     weight_decay=0.01, # regularization in optimizer
#     num_train_epochs=3,
#     predict_with_generate= True,
#     lr_scheduler_type="linear",
#     warmup_steps= 200 # (4000 samples/8 samples per batch) * 5 epochs
# )

In [20]:
# build trainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# trainer.train()
#untruncated version
# trainer.save_model('multi_news_train_800_t5-small_summary_model')

In [ ]:
# trainer.train()
#untruncated version
# trainer.save_model('multi_news_train_4000_t5-small_summary_model')

In [ ]:
# trainer.train()
#truncated version
# trainer.save_model('multi_news_train_4000t_t5-small_summary_model')

In [ ]:
# # attempted version
# trainer.train()
# trainer.save_model('multi_news_train_4000arg_t5-small_summary_model')

Step,Training Loss,Validation Loss


# **Login to HuggingFace**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub('multi_news_train_4000arg_t5-small_summary_model')

In [ ]:
document = ds_full['test'][0]['document']
test_summary_example = ds_full['test'][0]['summary']
test_summary_example

In [ ]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("multi_news_train_4000arg_t5-small_summary_model", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("multi_news_train_4000arg_t5-small_summary_model", local_files_only=True)

# Tokenize the input text
inputs = tokenizer(document, return_tensors="pt", max_length=1024, truncation=True)

# Generate summary
summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=None, max_length= 500, early_stopping=True)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
summary